In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score
from pathlib import Path
import joblib

# --- Configuration ---
OUTPUT_DIR = Path("../output")
DATA_FILE = OUTPUT_DIR / "readmissions_dataset.parquet"
MODEL_DIR = Path("../models")
MODEL_DIR.mkdir(exist_ok=True) # Create models directory if it doesn't exist

# --- Plotting Style ---
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

# 1. Data Loading

In [2]:
df = pd.read_parquet(DATA_FILE)

print(f"Dataset loaded with {df.shape[0]:,} rows and {df.shape[1]} columns.")
print("\nFirst 5 rows:")
display(df.head())

print("\nData Info:")
df.info()

Dataset loaded with 104,067 rows and 21 columns.

First 5 rows:


,encounter_id,patient_id,readmitted_within_30_days,length_of_stay,age_at_admission,gender,race,marital_status,admission_reason,admission_reason_detail,...,total_claim_cost,income,admission_day_of_week,primary_diagnosis_code,provider_id,payer_dx_interaction,prior_admissions_last_year,num_diagnoses,num_procedures,num_medications
0,fd70210f-52cb-77ef-61c7-aadbd1018a99,e3d3ed64-ca29-b95b-dbba-abfa3ee61b44,0,1,69,male,White,D,Hospital admission (procedure),Acute non-ST segment elevation myocardial infa...,...,40610.128906,33853,Sunday,401314000,us-npi|9999868992,Medicare_401314000,0,1.0,0.0,4.0
1,f2fa6d45-a393-6aea-83d4-166fbaa0a4a9,34270d54-36d4-f26b-4e4d-0cdb327cf159,1,1,57,male,White,D,Hospital admission (procedure),Abnormal findings diagnostic imaging heart+cor...,...,37202.929688,30294,Thursday,274531002,us-npi|9999868992,Aetna_274531002,0,1.0,21.0,0.0
2,3c16cdb1-a750-7963-b963-cc0f90e727f1,34270d54-36d4-f26b-4e4d-0cdb327cf159,1,2,57,male,White,D,Patient transfer to intensive care unit (proce...,History of coronary artery bypass grafting (si...,...,15647.690430,30294,Thursday,399261000,us-npi|9999868992,Aetna_399261000,1,0.0,6.0,0.0
3,3fc36a47-cb8f-56a4-3b11-a434126d40ca,34270d54-36d4-f26b-4e4d-0cdb327cf159,0,1,57,male,White,D,Admission to ward (procedure),History of coronary artery bypass grafting (si...,...,2176.229980,30294,Saturday,399261000,us-npi|9999868992,Aetna_399261000,2,0.0,4.0,0.0
4,df4e5b66-ca7f-5b68-a5c2-c77def0a94ce,13aebc7f-8e4a-396d-1666-1af41cb27781,0,13,43,male,White,S,Drug rehabilitation and detoxification (regime...,Dependent drug abuse (disorder),...,119.830002,11631,Monday,6525002,us-npi|9999927293,Anthem_6525002,0,0.0,0.0,0.0



Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104067 entries, 0 to 104066
Data columns (total 21 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   encounter_id                104067 non-null  object 
 1   patient_id                  104067 non-null  object 
 2   readmitted_within_30_days   104067 non-null  int32  
 3   length_of_stay              104067 non-null  int64  
 4   age_at_admission            104067 non-null  int64  
 5   gender                      104067 non-null  object 
 6   race                        104067 non-null  object 
 7   marital_status              104067 non-null  object 
 8   admission_reason            104067 non-null  object 
 9   admission_reason_detail     104062 non-null  object 
 10  payer                       104067 non-null  object 
 11  total_claim_cost            104067 non-null  float32
 12  income                      101108 non-null  Int64  
 13  ad

# 2. Feature Selection and Data Splitting

In [6]:
# Define target and features
TARGET = 'readmitted_within_30_days'
# Dropping ID columns and high-cardinality text columns for this initial model
features = [col for col in df.columns if col not in [TARGET, 'encounter_id', 'patient_id', 'admission_reason', 'admission_reason_detail']]

X = df[features]
y = df[TARGET]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Training set shape: {X_train.shape}")
print(f"Testing set shape: {X_test.shape}")

Training set shape: (83254, 9)
Testing set shape: (20814, 9)


# 3. Preprocessing Pipeline

In [7]:
# Identify numeric and categorical features
numeric_features = X.select_dtypes(include=np.number).columns.tolist()
categorical_features = X.select_dtypes(exclude=np.number).columns.tolist()

print(f"Numeric features: {numeric_features}")
print(f"Categorical features: {categorical_features}")

# Create preprocessing pipelines for numeric and categorical data
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Create a column transformer to apply different transformations to different columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

Numeric features: ['length_of_stay', 'age_at_admission', 'prior_admissions_last_year', 'num_diagnoses', 'num_procedures', 'num_medications']
Categorical features: ['gender', 'race', 'marital_status']


# 4. Model Training

In [8]:
# Define the model
# Using class_weight='balanced' is a good practice for imbalanced datasets like this one.
model = LogisticRegression(class_weight='balanced', random_state=42, max_iter=1000)

# Create the full pipeline
full_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                ('classifier', model)])

# Train the model
print("Training the Logistic Regression model...")
full_pipeline.fit(X_train, y_train)
print("Training complete.")

Training the Logistic Regression model...
Training complete.


# 5. Model Evaluation

In [9]:
# Make predictions
y_pred = full_pipeline.predict(X_test)
y_pred_proba = full_pipeline.predict_proba(X_test)[:, 1]

# Evaluate the model
print("\\n--- Classification Report ---")
print(classification_report(y_test, y_pred))

print(f"--- ROC AUC Score ---")
print(f"{roc_auc_score(y_test, y_pred_proba):.4f}")

\n--- Classification Report ---
              precision    recall  f1-score   support

           0       0.94      0.74      0.83     17107
           1       0.39      0.78      0.52      3707

    accuracy                           0.75     20814
   macro avg       0.67      0.76      0.68     20814
weighted avg       0.84      0.75      0.77     20814

--- ROC AUC Score ---
0.8157


# 6. Save the Model

In [10]:
# Save the pipeline to a file
model_filename = MODEL_DIR / 'logistic_regression_pipeline_v1.joblib'
joblib.dump(full_pipeline, model_filename)

print(f"\\nModel pipeline saved to: {model_filename}")

\nModel pipeline saved to: ../models/logistic_regression_pipeline_v1.joblib


# 7. Gradient Boosting Model (XGBoost)

In [11]:
import xgboost as xgb

# --- XGBoost Model Definition ---

# Calculate the scale_pos_weight for handling class imbalance
# It's the ratio of negative class to positive class
scale_pos_weight = y_train.value_counts()[0] / y_train.value_counts()[1]

xgb_model = xgb.XGBClassifier(
    objective='binary:logistic',
    scale_pos_weight=scale_pos_weight, # Handles imbalance
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42
)

# Create the full pipeline with the XGBoost model
xgb_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('classifier', xgb_model)])

# Train the model
print("Training the XGBoost model...")
xgb_pipeline.fit(X_train, y_train)
print("Training complete.")

Training the XGBoost model...
Training complete.


/usr/local/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [20:45:53] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


# 8. XGBoost Model Evaluation

In [12]:
# Make predictions with the XGBoost pipeline
y_pred_xgb = xgb_pipeline.predict(X_test)
y_pred_proba_xgb = xgb_pipeline.predict_proba(X_test)[:, 1]

# Evaluate the model
print("\\n--- Classification Report (XGBoost) ---")
print(classification_report(y_test, y_pred_xgb))

print(f"--- ROC AUC Score (XGBoost) ---")
print(f"{roc_auc_score(y_test, y_pred_proba_xgb):.4f}")

\n--- Classification Report (XGBoost) ---
              precision    recall  f1-score   support

           0       0.98      0.79      0.88     17107
           1       0.49      0.92      0.64      3707

    accuracy                           0.81     20814
   macro avg       0.73      0.85      0.76     20814
weighted avg       0.89      0.81      0.83     20814

--- ROC AUC Score (XGBoost) ---
0.9265


# 9. Save the XGBoost Model

In [13]:
# Save the XGBoost pipeline to a file
model_filename_xgb = MODEL_DIR / 'xgboost_pipeline_v1.joblib'
joblib.dump(xgb_pipeline, model_filename_xgb)

print(f"\\nModel pipeline saved to: {model_filename_xgb}")

\nModel pipeline saved to: ../models/xgboost_pipeline_v1.joblib
